In [1]:
import numpy as np
import cv2
import pandas as pd
from google.colab.patches import cv2_imshow
# silent all warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
import warnings
warnings.filterwarnings('ignore')
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

In [2]:
# Drag both Digit testing and training zip into collab files, then run code cell
!unzip -q /content/Digit_training.zip -d ../
!unzip -q /content/Digit_testing.zip -d ../

In [3]:
import glob
from skimage import io

def imagesToArray(path):
  gg = []
  for file in glob.glob(path):
   a = cv2.imread(file)
   gg.append(a)
  return np.copy(gg)
# source: https://stackoverflow.com/questions/51857468/google-colab-how-to-loop-through-images-in-a-folder    

In [4]:
# Initialize arrays that holds each classification
test_sample = imagesToArray("/Digit_testing/*/*.*")
train_sample = imagesToArray("/Digit_training/*/*.*")

In [5]:
def meanValue(sample):
    imgs = []
    for images in range(0,len(sample)): # images
      feature_matrix = np.zeros((28,28)) 
      for arrays in range(0,len(sample[images])): # 28
          for px in range(0,len(sample[images][arrays])): # 28
              feature_matrix[arrays][px] = ((int(sample[images][arrays][px][0]) + int(sample[images][arrays][px][1]) + int(sample[images][arrays][px][2]))/3)
      imgs.append(feature_matrix) 
    return np.copy(imgs)


In [6]:
train_sample.shape

(5000, 28, 28, 3)

In [7]:
train = meanValue(train_sample)
test = meanValue(test_sample)

print(train.shape)
print(len(test[0]))
print(len(train[0]))

(5000, 28, 28)
28
28


In [8]:
# Fourier_response function
def fourier_response(folder):
  imgs = []
  for img in range(0,len(folder)):
    imgs.append(np.real(np.fft.fft2(folder[img])))  
  return np.copy(imgs)   

In [9]:
# Flatten the matrix
def flatten(sample):
  data = []
  for x in range(0,len(sample)):
    data.append(sample[x].flatten())
  return np.copy(data)

In [10]:
# Normalization of data
def normalization(data):
  imgs = []
  for img in range(0,len(data)): # 200
    imgs.append((data[img] - data[img].min())/ (data[img].max() - data[img].min()))
  
  return np.copy(imgs)

In [11]:
cv2_imshow(train[0])
cv2_imshow(test[0])

In [12]:
print(train.shape)
print(test.shape)

(5000, 28, 28)
(2000, 28, 28)


In [13]:
# Creating furior transform for training
f_r_train = fourier_response(train)

In [14]:
print(f_r_train.shape)
print(len(f_r_train))
cv2_imshow(f_r_train[0])

(5000, 28, 28)
5000


In [15]:
# Creating furior transform for testing
f_r_test = fourier_response(test)

In [16]:
print(f_r_test.shape)
print(len(f_r_test))
cv2_imshow(f_r_test[0])

(2000, 28, 28)
2000


In [17]:
# Flatten the samples after fourier
f_r_test_flat = flatten(f_r_test)
f_r_train_flat = flatten(f_r_train)

In [18]:
f_r_test[0].flatten().shape

(784,)

In [19]:
print(f_r_test_flat.shape)
print(f_r_train_flat.shape)

(2000, 784)
(5000, 784)


In [20]:
# Normalization of Furior transformation
f_r_test_norm = normalization(f_r_test_flat)
f_r_train_norm = normalization(f_r_train_flat)

In [21]:
print((f_r_test_norm.shape))
len(f_r_test_norm)

(2000, 784)


2000

In [22]:
print(f_r_train_norm.shape)
print(len(f_r_train_norm))

(5000, 784)
5000


In [23]:
cv2_imshow(train[0])
cv2_imshow(train[500])
cv2_imshow(train[1000])
cv2_imshow(train[1500])
cv2_imshow(train[2000])
cv2_imshow(train[2500])
cv2_imshow(train[3000])
cv2_imshow(train[3500])
cv2_imshow(train[4000])
cv2_imshow(train[4500])

In [24]:
cv2_imshow(test[0])
cv2_imshow(test[200])
cv2_imshow(test[400])
cv2_imshow(test[600])
cv2_imshow(test[800])
cv2_imshow(test[1000])
cv2_imshow(test[1200])
cv2_imshow(test[1400])
cv2_imshow(test[1600])
cv2_imshow(test[1800])

In [25]:
# Creating y values for train and test
y_pred = np.zeros((10,500))
y_pred[0].fill(4)
y_pred[1].fill(5)
y_pred[2].fill(0)
y_pred[3].fill(1)
y_pred[4].fill(6)
y_pred[5].fill(9)
y_pred[6].fill(8)
y_pred[7].fill(2)
y_pred[8].fill(7)
y_pred[9].fill(3)

y_pred = y_pred.reshape(-1,1)

y_pred_test = np.zeros((10,200))
y_pred_test[0].fill(4)
y_pred_test[1].fill(5)
y_pred_test[2].fill(0)
y_pred_test[3].fill(1)
y_pred_test[4].fill(6)
y_pred_test[5].fill(9)
y_pred_test[6].fill(8)
y_pred_test[7].fill(2)
y_pred_test[8].fill(7)
y_pred_test[9].fill(3)

y_pred_test = y_pred_test.reshape(-1,1) 

In [26]:
print(y_pred.shape)

(5000, 1)


In [27]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
def KERNEL_SVM(deg, x, y, x_test, y_test):
  classifier = SVC(kernel ='poly', degree = deg,random_state = 0,C=1)
  classifier.fit(x,y)
  clf_pred = classifier.predict(x_test)
  acc = (accuracy_score(y_test, clf_pred)*100)
  return acc

In [28]:
kern_1 = KERNEL_SVM(1,f_r_train_norm, y_pred, f_r_test_norm, y_pred_test)
kern_3 = KERNEL_SVM(3,f_r_train_norm, y_pred, f_r_test_norm, y_pred_test)
kern_5 = KERNEL_SVM(5,f_r_train_norm, y_pred, f_r_test_norm, y_pred_test)

f = open("Accuracy.txt", "w")

f.write("Kernel SVM accuracy with degree 1, no pca: " + str(kern_1) + 
        "\nKernel SVM accuracy with degree 3, no pca: " + str(kern_3) + 
        "\nKernel SVM accuracy with degree 5, no pca: " + str(kern_5))

f.close()

In [32]:
from sklearn.decomposition import PCA
pca = PCA(n_components=196)
pca.fit(f_r_train_norm)
f_r_train_pca = pca.transform(f_r_train_norm)
f_r_test_pca = pca.transform(f_r_test_norm)

In [33]:
f_r_test_pca_norm = normalization(f_r_test_pca)
f_r_train_pca_norm = normalization(f_r_train_pca) 

In [35]:

kern_1_pca = KERNEL_SVM(1,f_r_train_pca_norm, y_pred, f_r_test_pca_norm, y_pred_test)
kern_3_pca = KERNEL_SVM(3,f_r_train_pca_norm, y_pred, f_r_test_pca_norm, y_pred_test)
kern_5_pca = KERNEL_SVM(5,f_r_train_pca_norm, y_pred, f_r_test_pca_norm, y_pred_test)

f = open("Accuracy_pca.txt", "w")

f.write("Kernel SVM accuracy with degree 1, pca: " + str(kern_1_pca) + 
        "\nKernel SVM accuracy with degree 3, pca: " + str(kern_3_pca) + 
        "\nKernel SVM accuracy with degree 5, pca: " + str(kern_5_pca))

f.close()